In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np

# Define transformations for the CIFAR-10 images
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize the images
])

# Load the CIFAR-10 dataset
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define the neural network
class CIFAR10Classifier(nn.Module):
    def __init__(self):
        super(CIFAR10Classifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)  # Flatten the tensor
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Instantiate the model, define the loss function and optimizer
model = CIFAR10Classifier()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()  # Zero the parameter gradients
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}')

# Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total:.2f}%')


100.0%


Epoch 1/10, Loss: 1.248684080075699
Epoch 2/10, Loss: 0.8546256343104179
Epoch 3/10, Loss: 0.6515401940237462
Epoch 4/10, Loss: 0.4706020337825621
Epoch 5/10, Loss: 0.3081719136164689
Epoch 6/10, Loss: 0.1855155334474849
Epoch 7/10, Loss: 0.12479248436397837
Epoch 8/10, Loss: 0.09619133017910436
Epoch 9/10, Loss: 0.07791392157658379
Epoch 10/10, Loss: 0.07303504503612279
Accuracy: 71.72%


In [2]:
# Save the model
torch.save(model.state_dict(), 'cifar10_model.pth')
print("Model saved to cifar10_model.pth")


Model saved to cifar10_model.pth


In [3]:
# Load the model
# Define the neural network
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np

class CIFAR10Classifier(nn.Module):
    def __init__(self):
        super(CIFAR10Classifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)  # Flatten the tensor
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Instantiate the model, define the loss function and optimizer
model = CIFAR10Classifier()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
    
model = CIFAR10Classifier()  # Recreate the model structure
model.load_state_dict(torch.load('cifar10_model.pth'))
model.eval()  # Set the model to evaluation mode
print("Model loaded from cifar10_model.pth")

Model loaded from cifar10_model.pth


In [5]:
from PIL import Image

# Update the transform to include resizing to 32x32 pixels
transform = transforms.Compose([
    transforms.Resize((32, 32)),  # Resize image to 32x32
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Function to preprocess and predict a single image
def predict_image(image_path):
    # Load the image
    image = Image.open(image_path).convert('RGB')
    
    # Apply the updated transformations
    image = transform(image).unsqueeze(0)  # Add a batch dimension

    # Pass the image through the model
    model.eval()
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)

    # CIFAR-10 classes
    classes = [
        'airplane', 'automobile', 'bird', 'cat', 'deer',
        'dog', 'frog', 'horse', 'ship', 'truck'
    ]
    return classes[predicted.item()]

# Example usage
image_path = 'data/horse.jpg'  # Replace with the path to your image
predicted_class = predict_image(image_path)
print(f'Predicted Class: {predicted_class}')


Predicted Class: horse
